In [1]:
from selenium import webdriver as opcoesSelenium
from selenium.webdriver.common.by import By
from google.cloud import bigquery
from google.oauth2 import service_account
import pandas as pd
import time
from pandas_gbq import to_gbq

## Extracting data from multiple tabs

In [4]:
def initialize_browser():
    """
    Initialize the Selenium browser instance.

    :return: A maximized Chrome browser instance navigated to the SteamDB sales page.
    """
    browser = opcoesSelenium.Chrome()
    browser.maximize_window()
    browser.get("https://steamdb.info/sales/")
    return browser

def scroll_down_page(browser, num_scrolls=11):
    """
    Scroll down the page smoothly to load more content.

    :param browser: The Selenium browser instance.
    :param num_scrolls: The number of scrolls to perform. Default is 11.
    """
    window_height = browser.execute_script("return window.innerHeight;")
    scroll_increment = window_height // 2

    for _ in range(num_scrolls):
        time.sleep(0.5)
        browser.execute_script("window.scrollBy(0, {});".format(scroll_increment))
        time.sleep(0.5)
    
def extract_table_data(browser):
    """
    Extract data from the SteamDB sales table.

    :param browser: The Selenium browser instance.
    :return: A list of dictionaries containing the extracted data.
    """
    data = []

    while True:
        scroll_down_page(browser)

        table_element = browser.find_element(By.XPATH, '/html/body/div[4]/div[1]/div[2]/div[2]/div[2]/div[3]/div/div[2]/table')
        table_rows = table_element.find_elements(By.TAG_NAME, "tr")

        for row in table_rows[1:]:  # Skip header
            columns = row.find_elements(By.TAG_NAME, "td")

            if len(columns) < 9:  # Ensure there are enough columns
                continue

            game_name, description, history = extract_game_details(columns[2].text)
            discount = columns[3].text.strip()
            price = columns[4].text.strip()
            rating = columns[5].text.strip()
            release_date = columns[6].text.strip()
            offer_end = columns[7].text.strip()
            last_update = columns[8].text.strip()

            row_data = {
                "Game_Name": game_name,
                "Description": description,
                "History": history,
                "Discount": discount,
                "Price": price,
                "Rating": rating,
                "Release_Date": release_date,
                "Offer_End": offer_end,
                "Last_Update": last_update
            }

            data.append(row_data)

        next_button = browser.find_element(By.CSS_SELECTOR, 'button.dt-paging-button.next')
        
        if 'disabled' in next_button.get_attribute("class"):
            break

        next_button.click()

    return data

def extract_game_details(text):
    """
    Extract game details from the given text.

    :param text: The text containing game details.
    :return: A tuple containing the game name, description, and history.
    """
    parts = text.split('\n')
    if len(parts) >= 3:
        return parts[0], parts[1], parts[2]
    elif len(parts) == 2:
        return parts[0], parts[1], ""
    else:
        return parts[0], "", ""
    
def save_to_excel(data, filename='SteamDB.xlsx'):
    """
    Save the extracted data to an Excel file.

    :param data: The list of dictionaries containing the extracted data.
    :param filename: The name of the Excel file to save the data. Default is 'SteamDB.xlsx'.
    """
    df = pd.DataFrame(data)
    df.to_excel(filename, header=True, index=False)

def main():
    """
    Main function to initialize the browser, extract table data, save it to an Excel file, and close the browser.
    """
    browser = initialize_browser()
    browser.get("https://steamdb.info/sales/")

    data = extract_table_data(browser)
    save_to_excel(data)

    browser.quit()

if __name__ == "__main__":
    main()

## Importing data into google big-query

In [6]:
def load_credentials(json_path, scopes):
    """
    Load Google Cloud service account credentials from a JSON file.

    :param json_path: Path to the service account JSON file.
    :param scopes: Scopes for the Google Cloud services.
    :return: Credentials object.
    """
    credentials = service_account.Credentials.from_service_account_file(
        json_path,
        scopes=scopes
    )
    return credentials

def save_dataframe_to_bigquery(dataframe, destination_table, project_id, credentials):
    """
    Save a pandas DataFrame to a Google BigQuery table.

    :param dataframe: The DataFrame to be saved.
    :param destination_table: The destination table in BigQuery (format: 'dataset.table').
    :param project_id: The Google Cloud project ID.
    :param credentials: Google Cloud service account credentials.
    """
    dataframe.to_gbq(
        destination_table=destination_table,
        project_id=project_id,
        if_exists='replace',
        credentials=credentials
    )

def main():
    json_path = r'C:\Users\andre\OneDrive\Área de Trabalho\SteamDB\JSON\beanalytic-423720-83290fae1aff.json'
    scopes = ["https://www.googleapis.com/auth/bigquery"]
    destination_table = 'SteamDb.SteamDB'
    project_id = 'beanalytic-423720'
    
    # Load credentials
    credentials = load_credentials(json_path, scopes)
    
    df = pd.read_excel('SteamDB.xlsx') 
    
    # Save DataFrame to BigQuery
    save_dataframe_to_bigquery(df, destination_table, project_id, credentials)

if __name__ == "__main__":
    main()

100%|██████████| 1/1 [00:00<?, ?it/s]
